In [64]:
try:
    import tensorflow.compat.v1 as tf
    tf.disable_v2_behavior()
except ImportError:
    import tensorflow as tf
import numpy as np
from datetime import datetime
from tensorboard import notebook

### Definição de funções.

In [65]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    '''Apaga todos os nós adicionados ao grafo padrão.'''
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
def createLogDir(root_logdir="tf_logs"):
    '''Cria no do diretório para se armazenar a rede.'''
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    logdir = "{}/run-{}/".format(root_logdir, now)
    return logdir
    
def shuffle_batch(X, y, batch_size):
    '''Cria um novo mini-batch com amostras aleatórias a cada nova chamada.'''
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch
        
def neuron_layer(X, n_neurons, name, activation=None):
    '''Implementa uma camada da rede neural.
           X: entrada
           n_neurons: número de nós na camada.
           name: string com o nome da camada.
           activation: função de ativação (se None, não usa função de ativação.).
    '''
    # Escopo de nomes que irá conter todos os nós da camada.
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        # Inicialização de He.
        stddev = 2 / np.sqrt(n_inputs)
        # Descarta e retira novamente quaisquer amostras que sejam maiores do que dois desvios padrão da média.
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        # Tensor 2D (i.e., uma matriz) contendo todos os pesos de conexão entre cada entrada e cada nó da camada.
        W = tf.Variable(init, name="weights")
        # Vetor que armazena os valores dos pesos de bias de cada um dos nós da camada.
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        # Calcula a combinação linear de cada um dos nós da camada.
        z = tf.matmul(X, W) + b
        # Função de ativação.
        if activation=="relu":
            return tf.nn.relu(z)
        else:
            return z

### Download e pré-processamento da base de dados.

In [66]:
# Carrega a base de dados.
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

# Redimensionamento e escalonamento dos atributos (valores variando entre 0 e 1).
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0

y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

### Definição de parâmetros da rede neural.

In [67]:
reset_graph()

logDir = createLogDir(root_logdir="./mlp_logs")
print(logDir)

learning_rate = 0.01

# Imagens com 28 por 28 pixels, onde cada pixel será um atributo de entrada.
n_inputs = 28*28 # MNIST
# Nós das camadas ocultas.
n_hidden1 = 300
n_hidden2 = 100
# Nós da camada de saída.
n_outputs = 10

./mlp_logs/run-20211116121823/


### Definindo o grafo de computação.

In [68]:
# None: número de exemplos é variável. Ou seja, o tamanho do mini-batch pode ser alterado em tempo de execução.
# O placeholder X será a camada de entrada.
# A cada iteração de treinamento, ele transferir um novo mini-batch ao grafo de computação. 
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

# Cria-se duas camadas ocultas e a camada de saída.
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, "hidden1", activation="relu")
    hidden2 = neuron_layer(hidden1, n_hidden2, "hidden2", activation="relu")
    # logits é a saída da rede neural antes de passar pela função de ativação do tipo softmax.
    # Por razões de otimização, trataremos do cálculo do softmax posteriormente.
    logits = neuron_layer(hidden2, n_outputs, "outputs")

# Definição da função de custo.
with tf.name_scope("loss"):
    # Equivalente a aplicar a função de ativação do tipo softmax e depois computar a entropia cruzada.
    # Porém, é mais eficiente computacionalmente e lida com casos onde os logits são iguais a zero.
    # labels são números inteiros variando de 0 a Q-1.
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    # Calcula a entropia cruzada média.
    loss = tf.reduce_mean(xentropy, name="loss")

# Definição do otimizador.
with tf.name_scope("train"):
    # Instancia otimizador. 
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    # Minimize a função de custo, ou seja, a entropia cruzada média.
    training_op = optimizer.minimize(loss)

# Definição de como avaliar o modelo.
with tf.name_scope("eval"):
    # Retorna vetor booleano indicando se cada logit de maior valor corresponde à classe em y.
    correct = tf.nn.in_top_k(logits, y, 1)
    # Converte o vetor booleano em float e calcula a média, resultando na acurácia.
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

# Cria nó de inicialização das variáveis.
init = tf.global_variables_initializer()

# Cria nó para salvar o modelo.
saver = tf.train.Saver()

# Cria nó para avaliar o custo.
loss_summary = tf.summary.scalar('loss', loss)

# Cria nó para avaliar a acurácia.
acc_summary = tf.summary.scalar('accuracy', accuracy)

# Cria nó para escrever o custo/acurácia em um arquivo.
file_writer = tf.summary.FileWriter(logDir, tf.get_default_graph())

### Executando o grafo de computação.

In [69]:
n_epochs = 20
batch_size = 50

# Cria-se uma sessão.
with tf.Session() as sess:
    # Inicializa as variáveis globais.
    init.run()
    for epoch in range(n_epochs):
        # A cada nova iteração, a função shuffle_batch retorna um novo mini-btach com amostras aleatórias.
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            # Realiza o treinamento, ou seja, atualização dos pesos/bias.
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        # Calcula as acurácias de treinamento e validação.
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        
        # Calcula o custo.
        summary_str = loss_summary.eval(feed_dict={X: X_batch, y: y_batch})
        # Armazena o custo em arquivo.
        file_writer.add_summary(summary_str, epoch)        
        # Calcula o custo.
        acc_summary_str = acc_summary.eval(feed_dict={X: X_batch, y: y_batch})        
        # Armazena a acurácia em arquivo.
        file_writer.add_summary(acc_summary_str, epoch)
        
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    # Salva os parâmetros do modelo.
    save_path = saver.save(sess, logDir+"my_model_finalv1.ckpt")

0 Train accuracy: 0.92 Test accuracy: 0.9172
1 Train accuracy: 0.94 Test accuracy: 0.9338
2 Train accuracy: 0.96 Test accuracy: 0.943
3 Train accuracy: 0.9 Test accuracy: 0.9472
4 Train accuracy: 0.96 Test accuracy: 0.951
5 Train accuracy: 0.94 Test accuracy: 0.9552
6 Train accuracy: 1.0 Test accuracy: 0.9586
7 Train accuracy: 0.96 Test accuracy: 0.9624
8 Train accuracy: 0.96 Test accuracy: 0.9634
9 Train accuracy: 0.94 Test accuracy: 0.966
10 Train accuracy: 0.92 Test accuracy: 0.9676
11 Train accuracy: 1.0 Test accuracy: 0.9682
12 Train accuracy: 0.98 Test accuracy: 0.9682
13 Train accuracy: 1.0 Test accuracy: 0.9692
14 Train accuracy: 1.0 Test accuracy: 0.97
15 Train accuracy: 0.94 Test accuracy: 0.9734
16 Train accuracy: 1.0 Test accuracy: 0.9724
17 Train accuracy: 1.0 Test accuracy: 0.9722
18 Train accuracy: 1.0 Test accuracy: 0.9738
19 Train accuracy: 0.98 Test accuracy: 0.9744


### Usando o modelo treinado.

In [70]:
with tf.Session() as sess:
    # Carrega-se os parâmetros do modelo armazenado em disco.
    saver.restore(sess, logDir+"my_model_finalv1.ckpt")
    # Usa-se algumas imagens escalonadas para serem classificadas.
    X_new_scaled = X_test[:20]
    # Avaliamos o nó logits.
    Z = logits.eval(feed_dict={X: X_new_scaled})
    # Prediz a classe das imagens encontrando o índice do elemento com maior valor do vetor.
    # Se quiséssemos encontrar a probabilidade de cada classe, deveríamos usar a função softmax.
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./mlp_logs/run-20211116121823/my_model_finalv1.ckpt


In [71]:
print("Predicted classes:", y_pred)
print("Actual classes:   ", y_test[:20])

Predicted classes: [7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 3 4]
Actual classes:    [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]


In [72]:
# Load the TensorBoard notebook extension
#%load_ext tensorboard
%reload_ext tensorboard

In [73]:
%tensorboard --logdir=./mlp_logs

Reusing TensorBoard on port 6006 (pid 2800), started 10:39:59 ago. (Use '!kill 2800' to kill it.)